# Climate Engine Authentication

The general authentication flow involves authenticating with the [Climate Engine Authorization Server](https://auth.climateengine.com/docs) and obtaining an access token, and using that token to access protected endpoints by including it in the Authorization header.

Here are some important endpoints that will be used in this tutorial:

`POST https://auth.climateengine.com/api/v1/auth/jwt/token`: Obtain an access token using some credentials

`GET https://auth.climateengine.com/api/v1/users/me`: Get information about your account (access token required)

`POST https://auth.climateengine.com/api/v1/clients/`: Create a new api client tied to your account (access token required)



### Requirements

`requests` and `authlib` libraries will be used for code examples in this tutorial, to install:

In [ ]:
!pip install requests authlib

In [ ]:
import requests
from authlib.integrations.requests_client import OAuth2Session
from authlib.oauth2.rfc7523 import ClientSecretJWT

### Token Endpoint

An access token can be obtained through a `POST` request to `/api/v1/auth/jwt/token` endpoint. The token endpoint supports OAuth2 `password` and `client_credentials` grant types. *Access tokens only last ~15 minutes*

#### **password grant type:**

In order to get an access token with `password` grant type, do a `POST` request to `/api/v1/auth/jwt/token` endpoint with the POST parameters `grant_type=password`, `username=<your_email>` and `password=<your_password>`. On successful authentication, the response will include an access token that is valid for ~15 minutes, along with some expiration information. This grant type is useful for testing, but the recommended grant type to obtain access tokens for client applications is `client_credentials`.

(`client_id` and `client_secret` fields are not required for `password` grant)

#### **client_credentials grant type:**

In order to get an access token with `client_credentials` grant type, you use your previously generated `client_id` and `client_secret` (more on that later) through one of the supported methods below:

(These are included here as a reference and they will generally be automatically handled with an oauth client library that supports these authentication methods (authlib in this tutorial's case).)

(`grant_type=client_credentials` POST parameter must be included for all the methods below)

**`client_secret_basic`:** Provide `client_id` and `client_secret` as username and password in HTTP Basic authorization header.

**`client_secret_post`:** Provide `client_id` and `client_secret` as POST parameters

**`client_secret_jwt`:** Create a JWT as described in the [spec](https://openid.net/specs/openid-connect-core-1_0.html#ClientAuthentication) and sign it with `client_secret`. Set `client_assertion_type=urn:ietf:params:oauth:client-assertion-type:jwt-bearer` POST parameter, and include the JWT in `client_assertion` POST parameter.


### Obtain an access token and access a protected endpoint

We will use `requests` for making the requests here, but you can follow along using the [interactive api docs](https://auth.climateengine.com/docs).

In the [interactive api docs](https://auth.climateengine.com/docs), for endpoints that require authentication (e.g the endpoints under the users and clients sections), you can authenticate by clicking the padlock icon on the right and entering your username and password under OAuth2PasswordBearer in the modal that pops up (client_id and client_secret are not required). When the access token expires (and you get a 401 Unauthorized response), you need to logout and re-login using the same modal.


#### Get an access token

In order to get an access token, we will send a POST request to `/api/v1/auth/jwt/token` endpoint with our username and password and grant_type=password as form fields in the body.

(If you get a 401 Unauthorized response for the sections below because the access token is expired, you can renew it by re-running the cell below)

In [ ]:
username = "bennett@climateengine.com"
password = "hunter2"

body = {"grant_type": "password", "username": username, "password": password}
# note that we are using data= parameter instead of json=, because this endpoint expects x-www-form-urlencoded body
response = requests.post(
    "https://auth.climateengine.com/api/v1/auth/jwt/token", data=body
)
response_data = response.json()
print(f"status: {response.status_code}")
print(response_data)

access_token = response_data["access_token"]

#### Access a protected endpoint using the access token

In [ ]:
headers = {"Authorization": "Bearer " + access_token}
response = requests.get(
    "https://auth.climateengine.com/api/v1/users/me", headers=headers
)
print(f"status: {response.status_code}")
print(response.json())

### Create a `client` object, and use it to obtain access tokens automatically with authlib and requests

#### Create an OAuth2 client_id and client_secret

In order to create a client, we do an empty POST request to `/api/v1/clients` endpoint. `client_secret` is only shown once during client creation and can't be accessed later, so make sure to save it somewhere safe.

In [ ]:
response = requests.post(
    "https://auth.climateengine.com/api/v1/clients/", headers=headers
)
response_data = response.json()
print(f"status: {response.status_code}")
print(response_data)

client_id = response_data["client_id"]
client_secret = response_data["client_secret"]

#### Create an authlib OAuth2Session client
OAuth2Session is also a requests (library) session, so it can be used just like the requests session. When using the `client_credentials` grant type and once an access token is fetched, authlib will automatically check whether the access token is expired before making a request and fetch a new token as necessary for the remaining requests.

In [ ]:
token_endpoint = "https://auth.climateengine.com/api/v1/auth/jwt/token"


def client_secret_jwt_client():
    client = OAuth2Session(
        client_id,
        client_secret,
        token_endpoint=token_endpoint,
        grant_type="client_credentials",
        token_endpoint_auth_method=ClientSecretJWT(token_endpoint),
    )
    client.fetch_token()
    return client


client = client_secret_jwt_client()
print("Getting an access_token with client_secret_jwt...")
print(client.fetch_token())

#### Call a protected endpoints with the authlib `client`

In [ ]:
client = client_secret_jwt_client()

response = client.get("https://auth.climateengine.com/api/v1/users/me")
print(f"status: {response.status_code}")
print(response.json())

If you see a `status 200` above, this indicates that you have successfully authenticated to the API.  You can now authenticate using your `client_id` and `client_secret` in future scripts!